# Vision Constants Optimization

This notebook is made to optimize vision constants.

This is done by the following steps:
1. Define vision calculation functiosn - exactly the same to what are used on the bot. Check that the results produced are the same.
2. Record true values to the target and corresponding camera pixels as reference for optimization.
3. Run the optimization.
4. Test if the result is accurate.

In [1]:
# Define vision functions.
# All angles in radians, all distance in meters.
import math

focalLength = 751.2954296909647
targetHeight = 2.606675


def calculateDistanceToTargetMeters(cameraHeightMeters, targetHeightMeters, cameraPitchRadians, targetPitchRadians):
    return (targetHeightMeters - cameraHeightMeters) / math.tan(cameraPitchRadians + targetPitchRadians)


def estimateCameraToTargetTranslation(targetDistanceMeters, yaw):
    return [math.cos(yaw) * targetDistanceMeters, math.sin(yaw) * targetDistanceMeters]


def calculatePitch(offset, target, focal):
    return -math.degrees(math.atan((offset - target) / focal))


def calculateYaw(offset, target, focal):
    return math.degrees(math.atan((offset - target) / focal))


def solveCameraToGoalTranslation(corner, goalHeight, lenHeight, pitchAngle):
    x = corner[0]
    y = corner[1]

    pitch = math.radians(calculatePitch(y, 480/2.0, focalLength))
    yaw = math.radians(calculateYaw(x, 640 / 2.0, focalLength))

    rangeLength = calculateDistanceToTargetMeters(
        lenHeight, goalHeight, pitchAngle, pitch)
    return estimateCameraToTargetTranslation(rangeLength, yaw)[0]


def visionFunction(corner, len_height, horizontal):
    return solveCameraToGoalTranslation(corner, targetHeight, len_height, horizontal)


In [3]:
# Test Vision Function
# The bot at the time shows estimated distance to be around 2.2912, the same with what the vision functions outputs
visionFunction([285.6,303.8], 1.53, math.radians(30))

2.2912497243544836

In [4]:
# Recording all the measured true corresponding values as a reference for optimization
# [distance x, distance y, pixel x, pixel y]
true_values = [
    [1.0, 0.0, 100, 100],
    [1.1, 0.0, 120, 100],
    [1.2, 0.0, 140, 100],
    [1.3, 0.0, 145, 100],
    [1.4, 0.0, 150, 100],
    [1.5, 0.0, 250, 100],
    [1.6, 0.0, 250, 100],
    [1.7, 0.0, 250, 100],
    [1.8, 0.0, 250, 100],
    [1.9, 0.0, 250, 100],
    [2.0, 0.0, 250, 100],
    [2.1, 0.0, 250, 100],
    [2.2, 0.0, 250, 100],
    [2.3, 0.0, 250, 100],
    [2.4, 0.0, 250, 100],
    [2.5, 0.0, 250, 100],
    [2.6, 0.0, 250, 100],
    [2.7, 0.0, 250, 100],
    [2.8, 0.0, 250, 100],
    [2.9, 0.0, 250, 100],
    [3.0, 0.0, 250, 100],
    [3.1, 0.0, 250, 100],
    [3.2, 0.0, 250, 100],
    [3.3, 0.0, 250, 100],
    [3.4, 0.0, 250, 100],
    [3.5, 0.0, 250, 100]
]


In [16]:
# Define error function as the target function for optimization
def targetErrorFunction(params):
    len_height = params[0]
    horizontal = params[1]
    
    error_sum = 0.0
    for value in true_values:
        estimate = visionFunction([value[2], value[3]], len_height, horizontal)
        # Simple variance
        error = (estimate - value[0])**2
        error_sum += error
    return error_sum

In [17]:
# Test error function
targetErrorFunction([1,2])

310.3340308522494

In [21]:
import scipy.optimize as optimize

# Height, Angle To Horizontal
initial_guess = [1.53, math.radians(45.0)]
bounds = [(0.0, 2.0), (math.radians(0.0), math.radians(90.0))]
print(initial_guess)

# Run Optimization
result = optimize.minimize(targetErrorFunction, initial_guess, bounds=bounds)
if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    raise ValueError(result.message)

[1.53, 0.7853981633974483]
[0.95371456 0.44380072]
